In [1]:
#!pip install torch
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.autograd import Variable
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:

x_min=0
x_max=0.5
T_min=0
T_max=8
lr=0.01

In [3]:
#Normalization
def xhat(x):
  x=(x-x_min)/(x_max-x_min)
  return (x)

In [4]:
#Normalization
def that(t):
  t=(t-T_min)/(T_max-T_min)
  return(t)

In [5]:
# We consider Net as our solution u_theta(x,t)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hidden_layer1 = nn.Linear(2,5)
        self.hidden_layer2 = nn.Linear(5,5)
        self.hidden_layer3 = nn.Linear(5,5)
        self.hidden_layer4 = nn.Linear(5,5)
        self.hidden_layer5 = nn.Linear(5,5)
        self.hidden_layer6 = nn.Linear(5,5)
        self.output_layer = nn.Linear(5,1)

    def forward(self, x,t):
        x=xhat(x)
        t=that(t)
        inputs = torch.cat([x,t],axis=1)
        layer1_out = torch.sigmoid(self.hidden_layer1(inputs))
        layer2_out = torch.sigmoid(self.hidden_layer2(layer1_out))
        layer3_out = torch.sigmoid(self.hidden_layer3(layer2_out))
        layer4_out = torch.sigmoid(self.hidden_layer4(layer3_out))
        layer5_out = torch.sigmoid(self.hidden_layer5(layer4_out))
        layer6_out = torch.sigmoid(self.hidden_layer5(layer5_out))
        output = self.output_layer(layer6_out)
        return output

In [6]:
### Model
net = Net()
net = net.to(device)
mse_cost_function = torch.nn.MSELoss() # Mean squared error
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [7]:
## PDE as loss function. Physics of the problem
def f(x, t, net):
    u = net(x,t)
    u_x = torch.autograd.grad(u.sum(), x, create_graph=True)[0]
    u_t = torch.autograd.grad(u.sum(), t, create_graph=True)[0]
    u_xx = torch.autograd.grad(u_x.sum(), x, create_graph=True)[0]
    pde = 400/16*u_t-0.002505/0.08*u_xx+0.004008/2*u+278.4/4*torch.abs(u)*u
    return pde

In [8]:
## Data from Boundary Conditions Or the fidelity data
t_bc = np.random.uniform(low=0.0, high=8.0, size=(5,1))
x_bc = np.zeros((5,1))
# compute u based on BC
u_bc = np.cos(np.pi/4*t_bc)

In [9]:
## Data from Boundary Conditions
t_bcf = np.random.uniform(low=0.0, high=8.0, size=(500,1))
x_bcf = np.ones((500,1))*0.2
# compute u based on BC
u_bcf = np.zeros((500,1))

In [10]:
### (3) Training / Fitting
iterations = 200000
loss_values=[]
matric_u=[]
m_loss=1
L_u=L_uf=L_f=0
for epoch in range(iterations):
    optimizer.zero_grad()
    lr= 1e-2 * (1e-4 / 1e-2) ** (epoch / 200000)
    # Loss based on boundary conditions
    pt_x_bc = Variable(torch.from_numpy(x_bc).float(), requires_grad=False).to(device)
    pt_t_bc = Variable(torch.from_numpy(t_bc).float(), requires_grad=False).to(device)
    pt_u_bc = Variable(torch.from_numpy(u_bc).float(), requires_grad=False).to(device)

    net_bc_out = net(pt_x_bc, pt_t_bc) # output of u(x,t)
    mse_u = mse_cost_function(net_bc_out, pt_u_bc)
    mse_u2= mse_u.cpu().detach().numpy()
    matric_u.append(mse_u2)  # to see changes in loss based on BC

# Loss based on boundary conditions 2
    pt_x_bcf = Variable(torch.from_numpy(x_bcf).float(), requires_grad=False).to(device)
    pt_t_bcf = Variable(torch.from_numpy(t_bcf).float(), requires_grad=False).to(device)
    pt_u_bcf = Variable(torch.from_numpy(u_bcf).float(), requires_grad=False).to(device)

    net_bc_outf = net(pt_x_bcf, pt_t_bcf) # output of u(x,t)
    mse_uf = mse_cost_function(net_bc_outf, pt_u_bcf)

    x_collocation = np.random.uniform(low=0.0, high=0.2, size=(500,1))
    t_collocation = np.random.uniform(low=0.0, high=8.0, size=(500,1))
    all_zeros = np.zeros((500,1))

    pt_x_collocation = Variable(torch.from_numpy(x_collocation).float(), requires_grad=True).to(device)
    pt_t_collocation = Variable(torch.from_numpy(t_collocation).float(), requires_grad=True).to(device)
    pt_all_zeros = Variable(torch.from_numpy(all_zeros).float(), requires_grad=False).to(device)

    f_out = f(pt_x_collocation, pt_t_collocation, net) # output of f(x,t)
    mse_f = mse_cost_function(f_out, pt_all_zeros)

    loss =mse_u+mse_f+mse_uf
    loss.backward() # This is for computing gradients using backward propagation
    loss2=loss.cpu().detach().numpy()
    loss_values.append(loss2)

    optimizer.step() # This is equivalent to : theta_new = theta_old - alpha * derivative of J w.r.t theta

    print("e:{}, mse_u:{}, mse_uf:{}, mse_f:{}, Loss:{}".format(epoch, mse_u, mse_uf, mse_f, loss ))



e:0, mse_u:1.2793976068496704, mse_uf:0.3406635522842407, mse_f:562.227783203125, Loss:563.8478393554688
e:1, mse_u:1.195337176322937, mse_uf:0.286540150642395, mse_f:397.7733459472656, Loss:399.2552185058594
e:2, mse_u:1.1175845861434937, mse_uf:0.23809219896793365, mse_f:274.6373596191406, Loss:275.9930419921875
e:3, mse_u:1.0466159582138062, mse_uf:0.19547998905181885, mse_f:185.13101196289062, Loss:186.37310791015625
e:4, mse_u:0.9826604723930359, mse_uf:0.1586485654115677, mse_f:121.94190979003906, Loss:123.08321380615234
e:5, mse_u:0.9256866574287415, mse_uf:0.12733739614486694, mse_f:78.55974578857422, Loss:79.6127700805664
e:6, mse_u:0.8754262924194336, mse_uf:0.10112123936414719, mse_f:49.542869567871094, Loss:50.51941680908203
e:7, mse_u:0.8314327597618103, mse_uf:0.07946785539388657, mse_f:30.59775161743164, Loss:31.508651733398438
e:8, mse_u:0.7931461334228516, mse_uf:0.0617973729968071, mse_f:18.503700256347656, Loss:19.358644485473633
e:9, mse_u:0.759955883026123, mse_uf:

KeyboardInterrupt: ignored

In [ ]:
plt.plot(np.array(loss_values), 'r')


In [ ]:
plt.plot(np.array(matric_u), 'k')

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()

t=np.arange(0.0, 8.0, 0.016)
t = t.reshape( len(t), -1 )
x=np.ones((500,1))*0

pt_x = Variable(torch.from_numpy(x).float(), requires_grad=False).to(device)
pt_t = Variable(torch.from_numpy(t).float(), requires_grad=False).to(device)
pt_u = net(pt_x,pt_t)
u=pt_u.data.cpu().numpy()
#ms_u = u.reshape(ms_x.shape)

plt.plot(t,u)


In [ ]:
###################################t=1
#x=np.random.uniform(low=0.0, high=0.5, size=(500,1))
x=np.arange(0, 0.5,0.001 )
x = x.reshape( len(x), -1 )

t=np.ones((500,1))*1

pt_x = Variable(torch.from_numpy(x).float(), requires_grad=False).to(device)
pt_t = Variable(torch.from_numpy(t).float(), requires_grad=False).to(device)
pt_u = net(pt_x,pt_t)
u=pt_u.data.cpu().numpy()

plt.plot(x,u)

In [ ]:
#########################t=2
x=np.arange(0, 0.5,0.001 )
x = x.reshape( len(x), -1 )
t=np.ones((500,1))*2

pt_x = Variable(torch.from_numpy(x).float(), requires_grad=False).to(device)
pt_t = Variable(torch.from_numpy(t).float(), requires_grad=False).to(device)
pt_u = net(pt_x,pt_t)
u=pt_u.data.cpu().numpy()
#ms_u = u.reshape(ms_x.shape)

plt.plot(x,u)


In [ ]:
##########################t=3
x=np.arange(0, 0.5,0.001 )
x = x.reshape( len(x), -1 )
t=np.ones((500,1))*3

pt_x = Variable(torch.from_numpy(x).float(), requires_grad=False).to(device)
pt_t = Variable(torch.from_numpy(t).float(), requires_grad=False).to(device)
pt_u = net(pt_x,pt_t)
u=pt_u.data.cpu().numpy()
#ms_u = u.reshape(ms_x.shape)

plt.plot(x,u)

In [ ]:
##########################t=4
x=np.arange(0, 0.5,0.001 )
x = x.reshape( len(x), -1 )
t=np.ones((500,1))*4

pt_x = Variable(torch.from_numpy(x).float(), requires_grad=False).to(device)
pt_t = Variable(torch.from_numpy(t).float(), requires_grad=False).to(device)
pt_u = net(pt_x,pt_t)
u=pt_u.data.cpu().numpy()
#ms_u = u.reshape(ms_x.shape)
plt.plot(x,u)

In [ ]:
##########################t=5
x=np.arange(0, 0.5,0.001 )
x = x.reshape( len(x), -1 )
t=np.ones((500,1))*5
#ms_x, ms_t = np.meshgrid(x, t)
## Just because meshgrid is used, we need to do the following adjustment
#x = np.ravel(ms_x).reshape(-1,1)
#t = np.zer(ms_t).reshape(-1,1)

pt_x = Variable(torch.from_numpy(x).float(), requires_grad=False).to(device)
pt_t = Variable(torch.from_numpy(t).float(), requires_grad=False).to(device)
pt_u = net(pt_x,pt_t)
u=pt_u.data.cpu().numpy()
#ms_u = u.reshape(ms_x.shape)

plt.plot(x,u)

In [ ]:
##########################t=6
x=np.arange(0, 0.5,0.001 )
x = x.reshape(len(x), -1 )
t = np.ones((500,1))*6

pt_x = Variable(torch.from_numpy(x).float(), requires_grad=False).to(device)
pt_t = Variable(torch.from_numpy(t).float(), requires_grad=False).to(device)
pt_u = net(pt_x,pt_t)
u=pt_u.data.cpu().numpy()
plt.plot(x,u)

In [ ]:
##########################t=7
x=np.arange(0, 0.5,0.001 )
x = x.reshape( len(x), -1 )
t=np.ones((500,1))*7
#ms_x, ms_t = np.meshgrid(x, t)
## Just because meshgrid is used, we need to do the following adjustment
#x = np.ravel(ms_x).reshape(-1,1)
#t = np.zer(ms_t).reshape(-1,1)

pt_x = Variable(torch.from_numpy(x).float(), requires_grad=False).to(device)
pt_t = Variable(torch.from_numpy(t).float(), requires_grad=False).to(device)
pt_u = net(pt_x,pt_t)
u=pt_u.data.cpu().numpy()
#ms_u = u.reshape(ms_x.shape)

plt.plot(x,u)

In [ ]:
###########################t=8
x=np.arange(0, 0.5,0.001 )
x = x.reshape( len(x), -1 )
t=np.ones((500,1))*8
#ms_x, ms_t = np.meshgrid(x, t)
## Just because meshgrid is used, we need to do the following adjustment
#x = np.ravel(ms_x).reshape(-1,1)
#t = np.zer(ms_t).reshape(-1,1)

pt_x = Variable(torch.from_numpy(x).float(), requires_grad=False).to(device)
pt_t = Variable(torch.from_numpy(t).float(), requires_grad=False).to(device)
pt_u = net(pt_x,pt_t)
u=pt_u.data.cpu().numpy()
#ms_u = u.reshape(ms_x.shape)

plt.plot(x,u)


In [ ]:
# Save Model
torch.save(net.state_dict(), "model_uxt.pt")

In [ ]:
!pip install mpl_toolkits.mplot3d
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
#ax = fig.gca(projection='3d')
ax = fig.add_subplot(111, projection='3d')
x=np.arange(0,1000,0.5)
t=np.arange(0,4,1)
ms_x, ms_t = np.meshgrid(x, t)
## Just because meshgrid is used, we need to do the following adjustment
x = np.ravel(ms_x).reshape(-1,1)
t = np.ravel(ms_t).reshape(-1,1)

pt_x = Variable(torch.from_numpy(x).float(), requires_grad=False).to(device)
pt_t = Variable(torch.from_numpy(t).float(), requires_grad=False).to(device)
pt_u = net(pt_x, pt_t)
u=pt_u.data.cpu().numpy()
x=pt_x.data.cpu().numpy()
t=pt_t.data.cpu().numpy()
forwardout={"y":x, "t":t, "u_pred":u}

savemat('testdata.mat', forwardout)
#torch.save((pt_x, pt_t, pt_u),'tensor.csv')

np.savetxt('output_u.csv', u)
np.savetxt('output_x.csv', x)
np.savetxt('output_t.csv', t)
ms_u = u.reshape(ms_x.shape)

if ax.name == '3d':
    surf = ax.plot_surface(ms_x,ms_t,ms_u, cmap=cm.coolwarm, linewidth=0, antialiased=False)